In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
titanic = pd.read_csv("tested.csv")

In [3]:
titanic = titanic.drop(["PassengerId", "Name", "Cabin","Ticket"], axis=1)

In [4]:
X = titanic.drop(["Survived"], axis=1)

In [5]:
y = titanic["Survived"]

In [6]:
X["Age"] = X["Age"].fillna(titanic["Age"].median())
X["Embarked"] = X["Embarked"].fillna(titanic["Embarked"].mode()[0])
X["Fare"] = X["Fare"].fillna(titanic["Fare"].median())


In [7]:
X = pd.get_dummies(X, columns=["Sex", "Embarked"], drop_first=False)

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [11]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) 
    return acc

In [43]:
class Titanic(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=10, out_features=15)
        self.layer2 = nn.Linear(in_features=15, out_features=15)
        self.layer3 = nn.Linear(in_features=15, out_features=1)

    def forwad(self, x):
        return self.layer3(self.layer2(self.layer1(x)))

In [44]:
model = Titanic()

In [45]:
model

Titanic(
  (layer1): Linear(in_features=10, out_features=15, bias=True)
  (layer2): Linear(in_features=15, out_features=15, bias=True)
  (layer3): Linear(in_features=15, out_features=1, bias=True)
)

In [46]:
model = nn.Sequential(
    nn.Linear(in_features=10, out_features=15),
    nn.Linear(in_features=15, out_features=15),
    nn.Linear(in_features=15, out_features=1)
)

In [47]:
model

Sequential(
  (0): Linear(in_features=10, out_features=15, bias=True)
  (1): Linear(in_features=15, out_features=15, bias=True)
  (2): Linear(in_features=15, out_features=1, bias=True)
)

In [50]:
loss_fn = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.SGD(params=model.parameters(), lr= 0.01)

In [51]:
torch.manual_seed(42)

epochs = 1000

for epoch in range(epochs):
    model.train()

    # forward pass
    y_logit = model(X_train)
    # turn logits to pred probs to pred labels
    y_pred = torch.round(torch.sigmoid(y_logit))
        
    # calculate the loss
    loss = loss_fn(y_logit, y_train)
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

    # optimizer zero grad
    optimizer.zero_grad()

    # loss backward
    loss.backward()

    # optimize step
    optimizer.step()

    ### TESTING  
    model.eval()
    with torch.inference_mode():
        # forward pass
        test_logits = model(X_test)
        test_pred = torch.round(torch.sigmoid(test_logits))

        # calculate the tesr loss
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)
        
    # print out
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss} | Accuracy {acc:.2%}| Test loss: {test_loss} | Test acc: {test_acc:.2f}%")

    

Epoch: 0 | Loss: 0.5164755582809448 | Accuracy 72.75%| Test loss: 0.5201010704040527 | Test acc: 0.77%
Epoch: 10 | Loss: 0.49479734897613525 | Accuracy 79.64%| Test loss: 0.4969012439250946 | Test acc: 0.85%
Epoch: 20 | Loss: 0.47172629833221436 | Accuracy 85.93%| Test loss: 0.4722973108291626 | Test acc: 0.92%
Epoch: 30 | Loss: 0.4472568929195404 | Accuracy 93.71%| Test loss: 0.4463144838809967 | Test acc: 0.98%
Epoch: 40 | Loss: 0.421477735042572 | Accuracy 98.50%| Test loss: 0.41907942295074463 | Test acc: 0.99%
Epoch: 50 | Loss: 0.39458218216896057 | Accuracy 98.80%| Test loss: 0.39082977175712585 | Test acc: 1.00%
Epoch: 60 | Loss: 0.36687007546424866 | Accuracy 99.10%| Test loss: 0.3619115352630615 | Test acc: 1.00%
Epoch: 70 | Loss: 0.33873695135116577 | Accuracy 99.40%| Test loss: 0.3327629268169403 | Test acc: 1.00%
Epoch: 80 | Loss: 0.3106473386287689 | Accuracy 100.00%| Test loss: 0.3038817346096039 | Test acc: 1.00%
Epoch: 90 | Loss: 0.2830953001976013 | Accuracy 100.00%| T